### Rondon's head_motion_prediction ingest and exploration

In [ ]:
import os
import sys
import plotly.express as px
import plotly.graph_objs as go
import numpy as np
from predict360user.head_motion_prediction import utils

project_path = f"predict360user/head_motion_prediction/"
sys.path.append(project_path)
from predict360user.head_motion_prediction.David_MMSys_18.Read_Dataset import (load_sampled_dataset, plot_3d_trace)

# show one traject
ONE_USER = '0'
ONE_VIDEO = '10_Cows'
if not 'SAMPLE_DATASET' in locals():
  print(f"get SAMPLE_DATASET from {project_path}")
  cwd = os.getcwd()
  os.chdir(project_path) # load_sampled_datasetuses relative path
  SAMPLE_DATASET = load_sampled_dataset()
  os.chdir(cwd)

traces = SAMPLE_DATASET[ONE_USER][ONE_VIDEO][:, 1:]

# plot 3d
plot_3d_trace(traces, ONE_USER, ONE_VIDEO)

# plot heatmap
heatmaps = []
for i in traces:
    heatmaps.append(
        utils.from_position_to_tile(
            utils.eulerian_in_range(*utils.cartesian_to_eulerian(i[0], i[1], i[2])),
            6,
            4,
        )
    )
fig = px.imshow(
    np.sum(heatmaps, axis=0),
    labels=dict(x="longitude", y="latitude", color="requests"),
    title=f"{len(traces)}_traces (reqs={np.sum(heatmaps)})",
)
fig.update_layout(go.Layout(width=600))
fig.show()


### predict360user: Ingest and exploration

In [ ]:
from predict360user import *

In [ ]:
df_trajecs = load_df_trajecs()(dataset_name="all")
print(f"number of trajecs in total {df_trajecs.size}")
print("\nnumber of trajecs per df_trajecs:") 
print(df_trajecs.reset_index()["df_trajecs"].value_counts())
print("\nnumber of trajecs per entropy class:")
print(df_trajecs["actS_c"].value_counts())
print("\ndf head")
display(df_trajecs.head())

In [ ]:
# show one traject
show_traject(df_trajecs.loc["david","0", "10_Cows"])

In [ ]:
# entropy distribution
show_entropy_histogram(df_trajecs)

In [ ]:
# show representive trajectory for each entropy class
show_trajects_representative(df_trajecs)

In [ ]:
split(df_trajecs)
# entropy distribution per partition
show_entropy_histogram_per_partition(df_trajecs)

### predict360user.Train

In [ ]:
from predict360user import *

In [ ]:
# load data and split
df_wins = load_df_wins(dataset_name="david")
df_wins = split(df_wins)

# fit model
cfg = Config()
model = build_model(cfg)
model.fit(df_wins)

# evaluate and log to wandb
err_per_class_dict = model.evaluate(model, df_wins)

In [ ]:
# show results
compare_train_results()
compare_eval_results()